**Black-Scholes model**

**1.1 Assumptions**

Black-Schloes model is one the most famous financial mathematics. It is often used by sell side of the market in order to find fair price of options.  
My variant of the model is based on the next assumptions:

1. The underlying asset of the option does not have dividents during the life of the option.

2. Price of the underlying asset is random.

3. No fee included into this model.

4. Risk-free rate and volatility are known and constant.

5. The returns of the underlying asset are normally distributed and prices have log-normal distribution (because they can not be negative)

6. The option can be only exercised at expiration.

**1.2 Analytical option pricing**

Assume that the price of underlying $S_t$ is following the stochastic differential equation[1]:

$dS_t = \mu S_tdt + \sigma S_tdW_t$

Where: 

$\mu$, $\sigma$ - constants;

$W_t$ - Wiener process.

Then according to the original paper [2]:

price of the option at $t$=0:

$C_0 = S_0\Phi(d_1) - e^{-rT}K\Phi(d_2)$      (1)

where:

$d_1=\frac{log(S_0/K)+(r+1/2\sigma^2)T}{\sigma\sqrt{T}}$


$d_2=\frac{log(S_0/K)+(r-1/2\sigma^2)T}{\sigma\sqrt{T}}$

$C_0$ - call option price at $t$=0;

$S_0$ - price of underlying at $t$=0;

$K$ - strike price;

$r$ - risk-free interest rate;

$T$ - time to maturity;

$\Phi$ - Cumulative normal distribution function.

Based on equation 1 $C_t$ at moment $t$ before maturity can be written as follows:

$C_t=(C_0/S_0)S_t$

Where $S_t$ - price of underlying at $t$.

**1.3 Price sensitivities (Greeks)**

Greeks are indicators of (local) sensitivity of an option price or the result of a portfolio strategy with respect to a given variable or parameter. This information is important for the computation of hedging strategies (for instance delta hedging), but also for risk management, optimization, and robustness analysis.[1]

Basic formulae for main Greeks:

$\Delta = \partial C / \partial S$ 

$\Gamma = \partial^2 C / \partial S^2$ 

$V = \partial C / \partial \sigma$ 

$\Theta = \partial C / \partial T$ 

$P = \partial C / \partial r$ 

Formulae for closed-form solution of the BS model:

For Call option:

$\Delta = \Phi(d_1)$ 

$\Gamma =\Phi(d_1) / (S_0 \sigma \sqrt{T})$ 

$V = S_0\Phi(d_1)\sqrt{T}$ 

$\Theta = -rKe^{-rT}\Phi(d_2) - \frac{S_0\Phi(d_1)\sigma}{2\sqrt{T}}$ 

$P = KTe^{-rT} \Phi(d2)$ 

For Put option:


$\Delta = -\Phi(-d_1)$ 

$\Gamma =\Phi(d_1) / (S_0 \sigma \sqrt{T})$ 

$V = S_0\Phi(d_1)\sqrt{T}$ 

$\Theta = rKe^{-rT}\Phi(-d_2) - \frac{S_0\Phi(d_1)\sigma}{2\sqrt{T}}$ 

$P = -KTe^{-rT} \Phi(-d2)$ 

Reference:
1. Schumacher, JM 2020, Introduction to Financial Derivatives. Open Press TiU.

2. Black, Fischer and Myron S. Scholes. “The Pricing of Options and Corporate Liabilities.” Journal of Political Economy 81 (1973): 637 - 654.

**Technical notes**

Specification for inputs for the model:

S0 - price of underlying at t=0, units per share;

r - risk-free rate (0.05 equals 5%);

sigma - volatility of underlying (0.3 equals 30% per year);

T - time to maturity (number of years);

K - strike price (same units as S0).

Output:

Option_object.Price() - price of the option in same units as S0 and K.

In [53]:
# import libraries
import scipy.stats as ss
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [54]:
class Black_Scholes (object):
  def __init__(self, S0,T,sigma, K,r,is_call):
    self.S=S0
    self.T=T
    self.sigma=sigma
    self.is_call=is_call
    self.r=r
    self.K=K
  def __Get_Probs(self):
    d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma**2) * self.T) / (self.sigma * np.sqrt(self.T))
    d2 = d1 - self.sigma * np.sqrt(self.T)
    self.d1 = d1
    self.d2 = d2
  def Price(self):
    self.__Get_Probs()
    if self.is_call:
      Opt_Price = self.S * ss.norm.cdf(self.d1) - self.K * np.exp(-self.r * self.T) * ss.norm.cdf(self.d2)
    else:
      Opt_Price = self.K * np.exp(-self.r * self.T) * ss.norm.cdf(-self.d2) - self.S * ss.norm.cdf(-self.d1)
    return Opt_Price
  def Greeks(self):
    self.__Get_Probs()
    Phi_d1=ss.norm.cdf(self.d1)
    if self.is_call:
      Delta = Phi_d1
      Gamma = Phi_d1 / (self.S * self.sigma * np.sqrt(self.T))
      Vega = self.S * Phi_d1 * np.sqrt(self.T)
      Theta = -(self.S * Phi_d1 * self.sigma) / (2 * np.sqrt(self.T)) - self.r * self.K * np.exp(-self.r * self.T) * ss.norm.cdf(self.d2)
      Rho = self.K * self.T * np.exp(-self.r * self.T) * ss.norm.cdf(self.d2)
    else:
      Delta = -ss.norm.cdf(-self.d1)
      Gamma = Phi_d1 / (self.S * self.sigma * np.sqrt(self.T))
      Vega = self.S * Phi_d1 * np.sqrt(self.T)
      Theta = -(self.S * Phi_d1 * self.sigma) / (2 * np.sqrt(self.T)) + self.r * self.K * np.exp(-self.r * self.T) * ss.norm.cdf(-self.d2)
      Rho = -self.K * self.T * np.exp(-self.r * self.T) * ss.norm.cdf(-self.d2)
    return Delta, Gamma, Vega, Theta, Rho
  @property
  def Type(self):
    string_ = str()
    if self.S==self.K:
      string_ += "ATM "
    string_ += "European"
    if self.is_call == True:
      string_ += " Call"
    else:
      string_ += " Put"
    return string_

In [55]:
#example
S0,T,sigma, K,r = 100, 2, 0.3, 100,0.06

params = (S0,T,sigma, K,r)

columns = {'Type':[], 'Price':[], 'Delta':[],'Gamma':[],'Vega':[],'Theta':[],'Rho':[]}
for is_call_ in [True,False]:
    Option_object = Black_Scholes (*params, is_call_)
    columns['Type'].append(Option_object.Type)
    columns['Price'].append(round(Option_object.Price(),2))

    D,G,V,T,R = Option_object.Greeks()
    columns['Delta'].append(D)
    columns['Gamma'].append(G)
    columns['Vega'].append(V)
    columns['Theta'].append(T)
    columns['Rho'].append(R)

pd.DataFrame(columns)

,Type,Price,Delta,Gamma,Vega,Theta,Rho
0,ATM European Call,22.12,0.689691,0.016256,97.53704,-10.126032,93.691790
1,ATM European Put,10.82,-0.310309,0.016256,97.53704,-4.804509,-83.692298
